In [1]:
from azureml.core import Workspace
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.conda_dependencies import CondaDependencies

In [2]:
## Dont Run below code again use saved configs

In [3]:
# ws = Workspace.create(
#                name='vehicular_coordination_deploy',            
#                subscription_id='c2f4dcb6-5111-4040-8a40-0c1777891f98',           
#                resource_group='vehicular_coordination5',                 
#                create_resource_group=True,                 
#                location='eastus'                
#                )

In [4]:
# ws.write_config()

In [5]:
ws = Workspace.from_config()

In [6]:
model = Model.register(workspace = ws,
              model_path ="lstmpicklemodel.pkl",
              model_name = "lstmPicklemodel",
              tags = {"version": "5"},
              description = "highd dataset lane Change Predictor",
              )
# path to model

Registering model lstmPicklemodel


In [11]:
#to install required packages
# to install required packages
env = Environment('env')
cd = CondaDependencies.create(pip_packages=['pynacl==1.2.1','numpy==1.19.5', 'azureml-defaults','torch==1.12.0','dill==0.3.5.1'],conda_packages=['pip==22.1.2'])
env.python.conda_dependencies = cd # Register environment to re-use later
env.register(workspace = ws)
print("Registered Environment")

Registered Environment


In [12]:
myenv = Environment.get(workspace=ws, name="env")

In [13]:
myenv.save_to_directory('./environ', overwrite=True)

Create a container instance and set the number of cpu_cores and memory_gb based on your requirements.

In [14]:
aciconfig = AciWebservice.deploy_configuration(
            cpu_cores=1,
            memory_gb=1,
            tags={"data":"HIGHD"},
            description='lane change prediction lstm model',
            )

Create and InferenceConfig instance to link your environment and entry script

### Local Deployment

In [15]:
# from azureml.core.webservice import LocalWebservice
# from azureml.core.model import InferenceConfig
# from azureml.core.environment import Environment
# from azureml.core import Workspace
# from azureml.core.model import Model

# ws = Workspace.from_config()
# model = Model(ws, 'lstmPicklemodel')


# inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

# deployment_config = LocalWebservice.deploy_configuration(port=6789)

# local_service = Model.deploy(workspace=ws, 
#                        name='lstm-highd-local', 
#                        models=[model], 
#                        inference_config=inference_config, 
#                        deployment_config = deployment_config)

# local_service.wait_for_deployment(show_output=True)
# print(f"Scoring URI is : {local_service.scoring_uri}")

In [16]:
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

## Deploy Model Online


Deploy the model by combining our config objects, workspace and model together. Deployment usually takes a few minutes.

If your deployment is successful, the rest endpoint will be printed out. If it is unsuccessful, you can access the deployment logs

In [ ]:
%%time
service = Model.deploy(workspace=ws,
                name='lstm-highd-model',
                models=[model],
                inference_config=inference_config,
                deployment_config=aciconfig, 
                overwrite = True)


service.wait_for_deployment(show_output=True)
url = service.scoring_uri
print(url)

In [ ]:
print(service.get_logs())

## Consume Local Service

In [ ]:
## Consume the service
import json

test_sample = json.dumps({'data': [
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]
]})

test_sample = bytes(test_sample, encoding='utf8')
prediction = service.run(input_data=test_sample)
print(prediction)

In [ ]:
!curl -p 127.0.0.1:5001/score